<small>
Copyright (c) 2017 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning From Basics to Practice
## by Andrew Glassner, https://dlbasics.com, http://glassner.com
------
## Chapter 27: Applications
### Notebook 1: Deep Dreams

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is still in the hacked-together form used to develop the figures, and is only lightly commented.

### About this code:
This notebook is adapted from
https://github.com/giuseppebonaccorso/keras_deepdream
by Giuseppe Bonaccorso.

See License D in LICENSE.txt

In [1]:
import keras.backend as K_backend
import multiprocessing
import warnings

from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.imagenet_utils import preprocess_input

from scipy.optimize import minimize

from skimage import img_as_float, img_as_ubyte
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage.transform import pyramid_gaussian, rescale

import numpy as np
import matplotlib.pyplot as plt

K_backend.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
# Because this notebook will generally be left alone
# for a long time, cranking out images, we won't use
# our file_helper here to save the images, since that 
# requires that the images be displayed in the 
# notebook. Instead we use imread and imsave
# from skimage.io. 
#
# Make a File_Helper for saving and loading files.

save_files = True

import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(current_dir)) # path to parent dir
from DLBasics_Utilities import File_Helper
file_helper = File_Helper(save_files)

# get the input and output directories
input_data_directory = file_helper.get_input_data_dir()
output_data_directory = file_helper.get_saved_output_dir()

# make sure the output directory exists
already_existed = file_helper.check_for_directory(output_data_directory)

In [3]:
# Ignore some warnings from scikit-image
warnings.simplefilter("ignore")

# Globals that get used when minimizing
loss_tensor = None
loss_gradient = None
_loss_function = None
_gradient_function = None

In [4]:
def show_image(image):
    fig, ax = plt.subplots(figsize=(18, 15))
    ax.imshow(image)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.show()
    
def preprocess_image(image):
    return preprocess_input(np.expand_dims(image.astype(K_backend.floatx()), 0))
    
def postprocess_image(image):
    image[:, :, :, 0] += 103.939
    image[:, :, :, 1] += 116.779
    image[:, :, :, 2] += 123.68
    return np.clip(image[:, :, :, ::-1], 0, 255).astype('uint8')[0]

In [5]:
def build_pyramid_from_image(original_image_array):
    # Create gaussian pyramid
    original_image_as_float = img_as_float(original_image_array)
    pyramid = list(pyramid_gaussian(original_image_as_float, downscale=2, max_layer=5))

    # Convert each image to unsigned byte (0-255)
    for i, image in enumerate(pyramid):
        pyramid[i] = img_as_ubyte(pyramid[i])
        print('Image {}) Size: {}'.format(i, pyramid[i].shape))
    return pyramid

In [6]:
# We'll use a big hammer and just use all the activations on a given
# layer with equal strength. It would be nice to apply per-filter weights
# before adding them up.
def make_loss_gradient_functions(convnet, chosen_layers, image_l2_weight):
    global loss_tensor, loss_gradient, _loss_function, _gradient_function
    
    loss_tensor = 0.0

    for layer, weight in chosen_layers.items():
        loss_tensor += (-weight * K_backend.sum(K_backend.square(convnet.get_layer(layer).output)))

    loss_tensor += image_l2_weight * K_backend.sum(K_backend.square(convnet.layers[0].input))

    _loss_function = K_backend.function(inputs=[convnet.layers[0].input], outputs=[loss_tensor])

    loss_gradient = K_backend.gradients(loss=loss_tensor, variables=[convnet.layers[0].input])
    _gradient_function = K_backend.function(inputs=[convnet.layers[0].input], outputs=loss_gradient)

In [7]:
def loss(x, shape):
    return _loss_function([x.reshape(shape)])[0]

def gradient(x, shape):
    return _gradient_function([x.reshape(shape)])[0].flatten().astype(np.float64)

In [8]:
def process_image(image, iterations=2):
    # Create bounds
    bounds = np.ndarray(shape=(image.flatten().shape[0], 2))
    bounds[:, 0] = -128.0
    bounds[:, 1] = 128.0

    # Initial value
    x0 = image.flatten()

    # Perform optimization
    result = minimize(fun=loss, 
                      x0=x0, 
                      args=list(image.shape), 
                      jac=gradient, 
                      method='L-BFGS-B', 
                      bounds=bounds, 
                      options={'maxiter': iterations})
    
    return postprocess_image(np.copy(result.x.reshape(image.shape)))

In [9]:
def build_dream(convnet, chosen_layers, image_l2_weight, original_image_array, do_resizing=False):

    if do_resizing:
        original_size = np.array(original_image_array).shape
        original_image_array = resize(original_image_array, (299, 299))
    pyramid = build_pyramid_from_image(original_image_array)

    make_loss_gradient_functions(convnet, chosen_layers, image_l2_weight)
    
    processed_image = None

    for i, image in enumerate(pyramid[::-1]):
        print('Processing pyramid image: {} {}'.format(len(pyramid)-i, image.shape))

        if processed_image is None:
            processed_image = process_image(preprocess_image(image))
        else:
            h, w = image.shape[0:2]
            ph, pw = processed_image.shape[0:2]
            rescaled_image = rescale(processed_image, order=5, scale=(float(h)/float(ph), float(w)/float(pw)))
            combined_image = img_as_ubyte((1.2*img_as_float(image) + 0.8*rescaled_image) / 2.0)
            processed_image = process_image(preprocess_image(combined_image), iterations=5)
            
    if do_resizing:
        processed_image = resize(processed_image, (original_size[0], original_size[1]))
    return processed_image

In [14]:
# Create visualization of the layers in VGG16
def visualize_layers():
    
    convnet = VGG16(include_top=False, weights='imagenet')
    np.random.seed(42)
    
    layer_names = [
        'block1_conv1', 'block1_conv2',
        'block2_conv1', 'block2_conv2',
        'block3_conv1', 'block3_conv2', 'block3_conv3',
        'block4_conv1', 'block4_conv2', 'block4_conv3',
        'block5_conv1', 'block5_conv2', 'block5_conv3',
    ]
    
    for name in layer_names:    
        original_image_array = np.random.uniform(low=-1, high=1, size=(224, 224, 3))
        layers_and_weights = { name : 0.001}
        original_image_l2_weight = 0
        do_resizing = False
        dream_image = build_dream(convnet, layers_and_weights, original_image_l2_weight,
                                  original_image_array, do_resizing)
        output_name = output_data_directory+'/only-layer-'+name+'.png'        
        imsave(output_name, dream_image)

In [ ]:
visualize_layers()

In [20]:
def stochastic_space_search_vgg_models(number_of_images_to_make=10000, random_seed=42):
    '''Generate random sets of variables and make deep-dream images.
    Generally, set number_of_images_to_make to some huge number, like 10000, and
    then leave the program running overnight (or for a weekend!). Then stop it
    manually when you have enough results.
      If you do leave this running for a long time, it might be worth the effort of
    adding a dictionary to keep track of which sets of parameters have been tried,
    so we don't accidentally repeat. 
      The variations are:
    * choice of image to start with
    * choice of network (VGG16 or VGG19 right now)
    * whether or not to scale image down to 299x299 for processing
    * the weight for the original image's L2 norm, when added into the loss
    * weights on the last 3 (or 4, if VGG19) convolution layers of the network
    '''
    image_list = [
        'waters-3038803_1280-crop', # frog
        'cat-2184682_1280',         # cat
        'dog-1210559_1280',         # dog
    ]

    print("loading convnets")
    x = imread('input_data/waters-3038803_1280-crop.jpg')
    print("x.shape=",x.shape)
    x0 = K_backend.zeros([1, x.shape[0], x.shape[1], x.shape[2]])
    convnet_dict = {
        'vgg16': VGG16(include_top=False, weights='imagenet'),
        'vgg19': VGG19(include_top=False, weights='imagenet'),
        #'resnet50': ResNet50(include_top=False, weights='imagenet'),
        #'xception': Xception(include_top=False, weights='imagenet'),
        #'inceptionv3': InceptionV3(include_top=False, weights='imagenet', input_tensor=x0),
        }
    print("done, all loaded.")
        
    np.random.seed(random_seed)
    
    for i in range(number_of_images_to_make):
        image_num = np.random.choice([0, 1, 2])
        convnet_size = np.random.choice([16, 19])
        convnet_name = 'vgg'+str(convnet_size)
        do_resizing = np.random.uniform() > 0.5    
        # These values are from trial and error. Feel free to change them.
        original_image_l2_weight = np.random.choice([0.001, 0.005, 0.00005]) 
        # w1 through w4 are the weights applied to the convolution layers
        # (the names match the layer names: _conv1, _conv2, etc.)
        w1 = np.random.choice([0, 1, 2, 5])
        w2 = np.random.choice([0, 1, 2, 5])
        w3 = np.random.choice([0, 1, 2, 5])
        w4 = np.random.choice([0, 1, 2, 5])
        if w1+w2+w3+w4 == 0:
            continue
        print("-------- New set of dream parameters -----")
        print("image_num=",image_num," name=",image_list[image_num])
        print("convnet_size=",convnet_size," convnet_name=",convnet_name)
        print("do_resizing=",do_resizing, "original_image_l2_weight=",original_image_l2_weight)
        print("w1=",w1," w2=",w2," w3=",w3," w4=",w4)
        print("--------")
        image_name = image_list[image_num]    
        convnet = convnet_dict[convnet_name]
        original_image = input_data_directory+'/'+image_name+'.jpg'
        original_image_array = imread(original_image)

        layer_set = {'layer_dict':{}, 'l2_weight': original_image_l2_weight }
        if w1 != 0:
            layer_set['layer_dict']['block5_conv1'] = w1 * 0.001
        if w2 != 0:
            layer_set['layer_dict']['block5_conv2'] = w2 * 0.001
        if w3 != 0:
            layer_set['layer_dict']['block5_conv3'] = w3 * 0.001
        if (convnet_size==19) and (w4 != 0):
            layer_set['layer_dict']['block5_conv4'] = w4 * 0.001
        layers_and_weights = layer_set['layer_dict']
        


        dream_image = build_dream(convnet, layers_and_weights, original_image_l2_weight,
                                  original_image_array, do_resizing)

        output_name = output_data_directory+'/Dream-'+convnet_name
        output_name += '-wgts-'+str(w1)+'_'+str(w2)+'_'+str(w3)+'_'+str(w4)
        output_name += '-image_l2_weight-'+str(original_image_l2_weight)
        output_name += '-resize-'+str(do_resizing)+'-'+image_name+'.png'
        
        imsave(output_name, dream_image)

In [ ]:
stochastic_space_search_vgg_models()